In [11]:
!pip install langchain_community
!pip install replicate

In [12]:
from langchain_community.llms import Replicate
from google.colab import userdata
import os

# Ambil api token dari colab
api_token = userdata.get("api_token")

# Kita masukin token ke env var
os.environ["REPLICATE_API_TOKEN"] = api_token

In [13]:
parameters = {
  "top_k": 5,
  "top_p": 0.1,
  "max_tokens": 4096,
  "min_tokens": 0,
  "random_seed": None,
  "repetition_penalty": 1.0,
  "temperature": 0.7,
  "stopping_criteria": "length (256 tokens)",
  "stopping_sequence": None
}

llm = Replicate(
  model="ibm-granite/granite-3.3-8b-instruct",
  input=parameters,
)


In [9]:
!pip install pandas
!pip install -U langchain_community langchain-experimental replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.7 MB/s eta 0:00:00


In [5]:
import pandas as pd
from zipfile import ZipFile

with ZipFile("archive (4).zip") as z:
    with z.open("Dataset Keuangan Truk Air Isi Ulang 2024.csv") as f:
        df = pd.read_csv(f)

df.head()


,No,Tanggal,Sopir,Plat Nomor,Order,Volume (L),Pemasukan,Pengeluaran,Jenis Transaksi,Jumlah,Keterangan
0,1,2024-06-01,Pak Joko,H 1111 AA,Depot Air Sehat,6000,"366,245.00",0,Pemasukan,"366,245.00",Pembayaran lunas
1,2,2024-06-01,Pak Dedi,H 2222 BB,Depot Air Sehat,8000,"360,609.00",0,Pemasukan,"360,609.00",Pembayaran lunas
2,3,2024-06-01,Pak Rudi,H 2222 BB,Warung Lesehan Berkah,10000,"367,988.00",0,Pemasukan,"367,988.00",Pembayaran lunas
3,4,2024-06-01,Pak Slamet,H 1234 CD,Ganti Oli,-,0.00,111296,Pengeluaran,"-111,296.00",Biaya ganti oli
4,5,2024-06-02,Pak Slamet,H 2222 BB,Warung Lesehan Berkah,10000,"313,188.00",0,Pemasukan,"313,188.00",Pembayaran lunas


In [20]:
from langchain_experimental.agents.agent_toolkits.pandas.base import (
    create_pandas_dataframe_agent
)
import pandas as pd
import matplotlib.pyplot as plt

agent = create_pandas_dataframe_agent (
    llm=llm,
    df=df,
    verbose=True,
    allow_dangerous_code=True,
)

In [38]:
agent.invoke("sopir dengan rata rata pendatapan paling banyak?")



> Entering new AgentExecutor chain...
Thought: The question asks for the driver with the highest average appearance. This requires calculating the average volume for each driver and then finding the driver with the maximum average.

Action: python_repl_ast

Action Input: 
```python
# First, calculate the average volume for each driver
avg_volume = df.groupby('Sopir')['Volume (L)'].mean().reset_index()

# Then, find the driver with the maximum average volume
max_avg_driver = avg_volume.loc[avg_volume['Volume (L)'].idxmax()]

max_avg_driver
```

Sopir            Pak Rudi
Volume (L)    8198.019802
Name: 6, dtype: objectThe driver with the highest average appearance in terms of volume is Pak Rudi with an average volume of approximately 8198.02 liters.

Final Answer: Pak Rudi has the highest average appearance with a volume of around 8198.02 liters.

> Finished chain.


{'input': 'sopir dengan rata rata pendatapan paling banyak?',
 'output': 'Pak Rudi has the highest average appearance with a volume of around 8198.02 liters.'}